# This Notebook contains examples on simulated data

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)
import random
import numpy as np
import pandas as pd
from synthetic_dgp.simulate import SIMULATE
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from models.ddfm import DDFM
import warnings
warnings.filterwarnings("ignore")

In [2]:
# seed setting
seed = 123456789
random.seed(seed)
np.random.seed(seed)
# set poly degree, if 1 then linear and the encoder structure is modified to linear single layer
poly_degree = 2
# init results storage
results_dfm = np.zeros(2)
results_ddfm = np.zeros(1)
# params sim data
n_obs = 50
r = 3
portion_missings = 0.2
# simulate data
sim_40 = SIMULATE(seed=seed, n=40, r=r, poly_degree=poly_degree)
sim_150 = SIMULATE(seed=seed, n=150, r=r, poly_degree=poly_degree)

## Simulate data 40

In [3]:
x = sim_40.simulate(n_obs, portion_missings=portion_missings)
r_true = sim_40.f.shape[1]
if poly_degree == 1:
    # For linear to set the following
    structure_encoder = (r_true,)
else:
    structure_encoder = (r_true * 6, r_true * 4, r_true * 2, r_true)

In [4]:
%%time
# estimate dfm
dyn_fact_mdl = DFM(pd.DataFrame(x), factors=min(r_true, x.shape[1]), factor_orders=1)
res_dyn_fact_mdl = dyn_fact_mdl.fit(disp=1000, maxiter=10000)

EM start iterations, llf=-1130.7
EM converged at iteration 530, llf=-950.24, convergence criterion=9.983e-07 < tolerance=1e-06
Wall time: 29.4 s


In [5]:
results_dfm[0] = sim_40.evaluate(res_dyn_fact_mdl.factors.smoothed.values, f_true=sim_40.f)
results_dfm[1] = sim_40.evaluate(res_dyn_fact_mdl.factors.filtered.values, f_true=sim_40.f)
print(results_dfm)

[0.7173566  0.71880755]


In [6]:
%%time
# estimate ddfm
deep_dyn_fact_mdl = DDFM(pd.DataFrame(x), structure_encoder=structure_encoder, factor_oder=1,
                             use_bias=False, link='relu', max_iter=1000)
deep_dyn_fact_mdl.fit()

@Info - Note: in data, last datapoint must be last observation.
@Info: iteration: 10 - new loss: 0.10883930229016198 - delta: 0.016920021173221708
@Info: Convergence achieved in 16 iterations!
Wall time: 1min 6s


In [7]:
results_ddfm[0] = sim_40.evaluate(np.mean(deep_dyn_fact_mdl.factors, axis=0), f_true=sim_40.f)
print(results_ddfm)

[0.90251958]



## Simulate data 150

In [8]:
x = sim_150.simulate(n_obs, portion_missings=portion_missings)
r_true = sim_150.f.shape[1]
if poly_degree == 1:
    # For linear to set the following
    structure_encoder = (r_true,)
else:
    structure_encoder = (r_true * 6, r_true * 4, r_true * 2, r_true)

In [9]:
%%time
# estimate dfm
dyn_fact_mdl = DFM(pd.DataFrame(x), factors=min(r_true, x.shape[1]), factor_orders=1)
res_dyn_fact_mdl = dyn_fact_mdl.fit(disp=1000, maxiter=1000)

EM start iterations, llf=-3418.7
EM converged at iteration 627, llf=-2945.7, convergence criterion=9.9993e-07 < tolerance=1e-06
Wall time: 6min 42s


In [10]:
results_dfm[0] = sim_150.evaluate(res_dyn_fact_mdl.factors.smoothed.values, f_true=sim_150.f)
results_dfm[1] = sim_150.evaluate(res_dyn_fact_mdl.factors.filtered.values, f_true=sim_150.f)
print(results_dfm)

[0.75016654 0.75034626]


In [11]:
%%time
# estimate ddfm
deep_dyn_fact_mdl = DDFM(pd.DataFrame(x), structure_encoder=structure_encoder, factor_oder=1,
                             use_bias=False, link='relu',
                         max_iter=1000)
deep_dyn_fact_mdl.fit()

@Info - Note: in data, last datapoint must be last observation.
@Info: iteration: 10 - new loss: 0.18475957298350976 - delta: 0.0018857126524865781
@Info: Convergence achieved in 14 iterations!
Wall time: 1min 10s


In [12]:
results_ddfm[0] = sim_150.evaluate(np.mean(deep_dyn_fact_mdl.factors, axis=0), f_true=sim_150.f)
print(results_ddfm)

[0.95233669]
